In [1]:
import pandas as pd,numpy as np

In [2]:
def feature_create(register,launch,video,activity):
    import pandas as pd,numpy as np
    #选取全部user_id
    ru=pd.DataFrame(register['user_id'])
    lu=pd.DataFrame(launch['user_id'])
    vu=pd.DataFrame(video['user_id'])
    au=pd.DataFrame(activity['user_id'])
    u1=ru.append(lu)
    u2=u1.append(vu)
    user_id=u2.append(au)
    user_id=user_id.drop_duplicates()
    #先提取特征，生成特征在提取特征之后
    #启动日志，启动天数
    launch_days=launch.groupby('user_id').size().reset_index().rename(columns={0:'launch_days'})
    #拍摄日志，拍摄次数
    video_times=video.groupby('user_id').size().reset_index().rename(columns={0:'video_times'})
    #行为日志
    #用户行为总数
    act_nums=activity.groupby('user_id').size().reset_index().rename(columns={0:'act_nums'})
    #用户涉及视频总数
    act_video_nums=activity.groupby('user_id')['video_id'].nunique().reset_index().rename(columns={'video_id':'act_video_nums'})
    #用户涉及作者总数
    act_authors=activity.groupby('user_id')['author_id'].nunique().reset_index().rename(columns={'author_id':'act_authors'})
    #同一个video的行为种类——>>平均每个video上的行为种类
    a_types=activity.groupby(['user_id','video_id'])['action_type'].nunique().reset_index()#rename(columns={'action_type':'act_types'})
    types_avg_video=a_types.groupby('user_id').agg({'action_type':np.mean}).reset_index().rename(columns={'action_type':'types_avg_video'})
    #新增属性
    #注册日志提取注册日期、注册类型
    drop=['device_type']
    rdf=register.drop(drop,axis=1)
    #启动日志，最近登录天数
    latest_day=launch.groupby('user_id').agg({'day':np.max}).reset_index().rename(columns={'day':'latest_day'})
    #行为日志，每天浏览页面种类平均数
    page_types=activity.groupby(['user_id','day'])['page'].nunique().reset_index().rename(columns={'page':'page_types'})
    daily_pagetypes=page_types.groupby('user_id').agg({'page_types':np.mean}).reset_index().rename(columns={'page_types':'daily_pagetypes'})
    #行为天数
    act_days=activity.groupby('user_id')['day'].nunique().reset_index().rename(columns={'day':'act_days'})
    #拍摄日志，拍摄天数
    video_days=video.groupby('user_id')['day'].nunique().reset_index().rename(columns={'day':'video_days'})
    #最大每日行为、拍摄数量
    eachday_acts=activity.groupby(['user_id','day']).size().reset_index().rename(columns={0:'eachday_acts'})
    max_daily_acts=eachday_acts.groupby('user_id').agg({'eachday_acts':np.max}).reset_index().rename(columns={'eachday_acts':'max_daily_acts'})
    eachday_videos=video.groupby(['user_id','day']).size().reset_index().rename(columns={0:'eachday_videos'})
    max_daily_videos=eachday_videos.groupby('user_id').agg({'eachday_videos':np.max}).reset_index().rename(columns={'eachday_videos':'max_daily_videos'})
    #单个视频、作者最大行为数
    acts_single_video=activity.groupby(['user_id','video_id']).size().reset_index().rename(columns={0:'acts_single_video'})
    maxacts_single_video=acts_single_video.groupby('user_id').agg({'acts_single_video':np.max}).reset_index().rename(columns={'acts_single_video':'maxacts_single_video'})
    acts_single_author=activity.groupby(['user_id','author_id']).size().reset_index().rename(columns={0:'acts_single_author'})
    maxacts_single_author=acts_single_author.groupby('user_id').agg({'acts_single_author':np.max}).reset_index().rename(columns={'acts_single_author':'maxacts_single_author'})
    #最后拍摄、行为日
    last_video_day=video.groupby('user_id').agg({'day':np.max}).reset_index().rename(columns={'day':'last_video_day'})
    last_act_day=activity.groupby('user_id').agg({'day':np.max}).reset_index().rename(columns={'day':'last_act_day'})
    #整合
    df1=pd.merge(user_id,launch_days,how='left',on=['user_id'])
    df2=pd.merge(df1,video_times,how='left',on=['user_id'])
    df3=pd.merge(df2,act_nums,how='left',on=['user_id'])
    df4=pd.merge(df3,act_video_nums,how='left',on=['user_id'])
    df5=pd.merge(df4,act_authors,how='left',on=['user_id'])
    df6=pd.merge(df5,rdf,how='left',on=['user_id'])
    df7=pd.merge(df6,latest_day,how='left',on=['user_id'])
    df8=pd.merge(df7,types_avg_video,how='left',on=['user_id'])
    df9=pd.merge(df8,act_days,how='left',on=['user_id'])
    df10=pd.merge(df9,video_days,how='left',on=['user_id'])
    df11=pd.merge(df10,daily_pagetypes,how='left',on=['user_id'])
    df12=pd.merge(df11,max_daily_acts,how='left',on=['user_id'])
    df13=pd.merge(df12,max_daily_videos,how='left',on=['user_id'])
    df14=pd.merge(df13,maxacts_single_video,how='left',on=['user_id'])
    df15=pd.merge(df14,maxacts_single_author,how='left',on=['user_id'])
    df16=pd.merge(df15,last_video_day,how='left',on=['user_id'])
    dataframe=pd.merge(df16,last_act_day,how='left',on=['user_id'])
    dataframe=dataframe.fillna(0)
    return dataframe

In [3]:
def feature_generate(df):
    #########7
    daily_video_times=df['video_times']/df['launch_days']
    daily_act_nums=df['act_nums']/df['launch_days']
    act_avg_video=df['act_nums']/df['act_video_nums']
    video_avg_author=df['act_video_nums']/df['act_authors']
    act_avg_author=df['act_nums']/df['act_authors']
    act_period=df['latest_day']-df['register_day']+1
    avg_launch_gap=act_period/df['launch_days']
    ###########10针对天数
    period=25-df['register_day']
    empty_days=period-df['launch_days']
    latest_gap=24-df['latest_day']
    empty_days_pct=empty_days/period
    latest_gap_pct=latest_gap/period
    empty_launch_rate=empty_days/df['launch_days']
    launch_days_rate=df['launch_days']/period
    latest_gap_launch=latest_gap/df['launch_days']
    latest_gap_empty=latest_gap/empty_days#出现nan
    period_ratio=act_period/period
    ####################6观察期行为
    video_times_pct=df['video_times']/period
    act_pct=df['act_nums']/period
    video_nums_pct=df['act_video_nums']/period
    authors_pct=df['act_authors']/period
    videodays_pct=df['video_days']/period
    actdays_pct=df['act_days']/period
    ############行为最大值比较
    s_vacts_r=df['maxacts_single_video']/df['act_nums']
    s_aacts_r=df['maxacts_single_author']/df['act_nums']
    m_d_acts=df['max_daily_acts']/daily_act_nums
    m_d_videos=df['max_daily_videos']/daily_video_times
    ############gap
    last_video_gap=df['latest_day']-df['last_video_day']
    last_act_gap=df['latest_day']-df['last_act_day']
    #7属性合并生成
    df.insert(19,'daily_video_times',daily_video_times)
    df.insert(20,'daily_act_nums',daily_act_nums)
    df.insert(21,'act_avg_video',act_avg_video)
    df.insert(22,'video_avg_author',video_avg_author)
    df.insert(23,'act_avg_author',act_avg_author)
    df.insert(24,'act_period',act_period)
    df.insert(25,'avg_launch_gap',avg_launch_gap)
    ##########10
    df.insert(26,'period',period)
    df.insert(27,'empty_days',empty_days)
    df.insert(28,'latest_gap',latest_gap)
    df.insert(29,'empty_days_pct',empty_days_pct)
    df.insert(30,'latest_gap_pct',latest_gap_pct)
    df.insert(31,'empty_launch_rate',empty_launch_rate)
    df.insert(32,'launch_days_rate',launch_days_rate)
    df.insert(33,'latest_gap_launch',latest_gap_launch)
    df.insert(34,'latest_gap_empty',latest_gap_empty)
    df.insert(35,'period_ratio',period_ratio)
    ###########6
    df.insert(36,'video_times_pct',video_times_pct)
    df.insert(37,'act_pct',act_pct)
    df.insert(38,'video_nums_pct',video_nums_pct)
    df.insert(39,'videodays_pct',videodays_pct)
    df.insert(40,'actdays_pct',actdays_pct)
    df.insert(41,'authors_pct',authors_pct)
    df.insert(42,'s_vacts_r',s_vacts_r)
    df.insert(43,'s_aacts_r',s_aacts_r)
    df.insert(44,'m_d_acts',m_d_acts)
    df.insert(45,'m_d_videos',m_d_videos)
    df.insert(46,'last_video_gap',last_video_gap)
    df.insert(47,'last_act_gap',last_act_gap)
    import numpy as np
    isinf=np.isinf(df)
    df[isinf]=0
    df=df.fillna(0)
    return df

In [4]:
def generate_test(df):
    #########7
    daily_video_times=df['video_times']/df['launch_days']
    daily_act_nums=df['act_nums']/df['launch_days']
    act_avg_video=df['act_nums']/df['act_video_nums']
    video_avg_author=df['act_video_nums']/df['act_authors']
    act_avg_author=df['act_nums']/df['act_authors']
    act_period=df['latest_day']-df['register_day']+1
    avg_launch_gap=act_period/df['launch_days']
    ###########10针对天数
    period=31-df['register_day']
    empty_days=period-df['launch_days']
    latest_gap=30-df['latest_day']
    empty_days_pct=empty_days/period
    latest_gap_pct=latest_gap/period
    empty_launch_rate=empty_days/df['launch_days']
    launch_days_rate=df['launch_days']/period
    latest_gap_launch=latest_gap/df['launch_days']
    latest_gap_empty=latest_gap/empty_days#出现nan
    period_ratio=act_period/period
    ####################6观察期行为
    video_times_pct=df['video_times']/period
    act_pct=df['act_nums']/period
    video_nums_pct=df['act_video_nums']/period
    authors_pct=df['act_authors']/period
    videodays_pct=df['video_days']/period
    actdays_pct=df['act_days']/period
    ############行为最大值比较
    s_vacts_r=df['maxacts_single_video']/df['act_nums']
    s_aacts_r=df['maxacts_single_author']/df['act_nums']
    m_d_acts=df['max_daily_acts']/daily_act_nums
    m_d_videos=df['max_daily_videos']/daily_video_times
    ############gap
    last_video_gap=df['latest_day']-df['last_video_day']
    last_act_gap=df['latest_day']-df['last_act_day']
    #7属性合并生成
    df.insert(19,'daily_video_times',daily_video_times)
    df.insert(20,'daily_act_nums',daily_act_nums)
    df.insert(21,'act_avg_video',act_avg_video)
    df.insert(22,'video_avg_author',video_avg_author)
    df.insert(23,'act_avg_author',act_avg_author)
    df.insert(24,'act_period',act_period)
    df.insert(25,'avg_launch_gap',avg_launch_gap)
    ##########10
    df.insert(26,'period',period)
    df.insert(27,'empty_days',empty_days)
    df.insert(28,'latest_gap',latest_gap)
    df.insert(29,'empty_days_pct',empty_days_pct)
    df.insert(30,'latest_gap_pct',latest_gap_pct)
    df.insert(31,'empty_launch_rate',empty_launch_rate)
    df.insert(32,'launch_days_rate',launch_days_rate)
    df.insert(33,'latest_gap_launch',latest_gap_launch)
    df.insert(34,'latest_gap_empty',latest_gap_empty)
    df.insert(35,'period_ratio',period_ratio)
    ###########6
    df.insert(36,'video_times_pct',video_times_pct)
    df.insert(37,'act_pct',act_pct)
    df.insert(38,'video_nums_pct',video_nums_pct)
    df.insert(39,'videodays_pct',videodays_pct)
    df.insert(40,'actdays_pct',actdays_pct)
    df.insert(41,'authors_pct',authors_pct)
    df.insert(42,'s_vacts_r',s_vacts_r)
    df.insert(43,'s_aacts_r',s_aacts_r)
    df.insert(44,'m_d_acts',m_d_acts)
    df.insert(45,'m_d_videos',m_d_videos)
    df.insert(46,'last_video_gap',last_video_gap)
    df.insert(47,'last_act_gap',last_act_gap)
    import numpy as np
    isinf=np.isinf(df)
    df[isinf]=0
    df=df.fillna(0)
    return df

In [5]:
def split_train_test(r,l,v,a):
    r_train=r[r['register_day']<=24]
    r_test=r[(r['register_day']>24)&(r['register_day']<31)]
    l_train=l[l['day']<=24]
    l_test=l[(l['day']>24)&(l['day']<31)]
    v_train=v[v['day']<=24]
    v_test=v[(v['day']>24)&(v['day']<31)]
    a_train=a[a['day']<=24]
    a_test=a[(a['day']>24)&(a['day']<31)]
    return r_train,r_test,l_train,l_test,v_train,v_test,a_train,a_test

In [6]:
def user_id_concat(r,l,v,a):
    ru=pd.DataFrame(r['user_id'])
    lu=pd.DataFrame(l['user_id'])
    vu=pd.DataFrame(v['user_id'])
    au=pd.DataFrame(a['user_id'])
    u1=ru.append(lu)
    u2=u1.append(vu)
    user_id=u2.append(au)
    user_id=user_id.drop_duplicates()
    return user_id

In [7]:
def train_add_feature(df):
    ind=df.index.max()+1
    label=np.zeros((ind,1))
    df.insert(48,'no_act_video',label)
    df.insert(49,'video_or_not',label)
    df.insert(50,'lately_r',label)
    df.insert(51,'long_r',label)
    df.insert(52,'lately_l',label)
    df.insert(53,'lately_a_v',label)
    df.insert(54,'bias_or_not',label)
    #拍摄行为是否为0
    d1=df[(df['video_times']==0)&(df['act_nums']==0)].index
    for i in d1:
        df.set_value(index=i,col='no_act_video',value=1)
    #拍摄是否为0
    d2=df[(df['video_times']>0)].index
    for i in d2:
        df.set_value(index=i,col='video_or_not',value=1)
    #是否最近注册
    d3=df[(df['register_day']>=20)].index
    for i in d3:
        df.set_value(index=i,col='lately_r',value=1)
    #是否很久之前注册
    d4=df[((df['register_day']<=5))].index
    for i in d4:
        df.set_value(index=i,col='long_r',value=1)
    #最近有无登录
    d5=df[df['latest_day']>=20].index
    for i in d5:
        df.set_value(index=i,col='lately_l',value=1)
    #最近有无拍摄、行为
    d6=df[(df['last_video_day']>=20)|(df['last_act_day']>=20)].index
    for i in d6:
        df.set_value(index=i,col='lately_a_v',value=1)
    #有无偏好、用户粘性
    d7=df[(df['video_avg_author']>=3)|(df['act_avg_author']>=3)|(df['s_vacts_r']>=0.3)|(df['s_aacts_r']>=0.5)].index
    for i in d7:
        df.set_value(index=i,col='bias_or_not',value=1)
    return df
        
       
        

In [8]:
def test_add_feature(df):
    ind=df.index.max()+1
    label=np.zeros((ind,1))
    df.insert(48,'no_act_video',label)
    df.insert(49,'video_or_not',label)
    df.insert(50,'lately_r',label)
    df.insert(51,'long_r',label)
    df.insert(52,'lately_l',label)
    df.insert(53,'lately_a_v',label)
    df.insert(54,'bias_or_not',label)
    #拍摄行为是否为0
    d1=df[(df['video_times']==0)&(df['act_nums']==0)].index
    for i in d1:
        df.set_value(index=i,col='no_act_video',value=1)
    #拍摄是否为0
    d2=df[(df['video_times']>0)].index
    for i in d2:
        df.set_value(index=i,col='video_or_not',value=1)
    #是否最近注册
    d3=df[(df['register_day']>=26)].index
    for i in d3:
        df.set_value(index=i,col='lately_r',value=1)
    #是否0~10天注册
    d4=df[((df['register_day']<=10))].index
    for i in d4:
        df.set_value(index=i,col='long_r',value=1)
    #最近有无登录
    d5=df[df['latest_day']>=26].index
    for i in d5:
        df.set_value(index=i,col='lately_l',value=1)
    #最近有无拍摄、行为
    d6=df[(df['last_video_day']>=26)|(df['last_act_day']>=26)].index
    for i in d6:
        df.set_value(index=i,col='lately_a_v',value=1)
    #有无偏好、用户粘性
    d7=df[(df['video_avg_author']>=3)|(df['act_avg_author']>=3)|(df['s_vacts_r']>=0.3)|(df['s_aacts_r']>=0.5)].index
    for i in d7:
        df.set_value(index=i,col='bias_or_not',value=1)
    return df

In [9]:
def get_max_lx(array):
    array=np.array(array)
    array.sort()
    k=np.diff(array)
    index=len(k)
    i=0
    days=0
    m=0
    while(i<=index-1):
        if(k[i]==1):
            days+=1
            i+=1
            m=max(days,m)
            if(i==index):
                break
        else:
            days=0
            i+=1
            if(i==index):
                break
    return m+1
def get_last_lx(a):
    a=np.array(a)
    a.sort()
    k=np.diff(a)
    index=len(k)
    i=0
    days=0
    m=0
    result=[]
    while(i<=index-1):
        while((k[i]==1)&(i<=index-1)):
            days=days+1
            i=i+1
            m=days
            if(i==index):
                break
        if(m>0):        
            result.append(m+1)
        m=0
        days=0
        i+=1
    last=len(result)-1
    if(last>=0):
        return result[last]
    else:
        return 1
def get_lx_var(a):
    a=np.array(a)
    a.sort()
    k=np.diff(a)
    index=len(k)
    i=0
    days=0
    m=0
    result=[]
    while(i<=index-1):
        while((k[i]==1)&(i<=index-1)):
            days=days+1
            i=i+1
            m=days
            if(i==index):
                break
        if(m>0):        
            result.append(m+1)
        m=0
        days=0
        i+=1
    r=np.array(result)
    if(len(r)<=1):
        return 0
    else:
        var=r.var()
        return var

In [10]:
def get_gap_times(a):
    a=np.array(a)
    index=len(a)
    count=0
    i=0
    for i in range(index):
        if(a[i]!=1):
            count=count+1
    return count

In [11]:
def last_lx_var(a):
    a=np.array(a)
    a.sort()
    k=np.diff(a)
    index=len(k)
    i=0
    days=0
    m=0
    result=[]
    while(i<=index-1):
        while((k[i]==1)&(i<=index-1)):
            days=days+1
            i=i+1
            m=days
            if(i==index):
                break
        if(m>0):        
            result.append(m+1)
        m=0
        days=0
        i+=1
    r=np.array(result)
    length=len(r)
    res=[]
    if(length<=1):
        return 0
    if(length>1):
        mean=r.mean()
        l=r[length-1]
        res.append(mean)
        res.append(l)
        res=np.array(res)
        var=res.var()
        return var

In [12]:
def quarter_lx_var(a):
    a=np.array(a)
    a.sort()
    k=np.diff(a)
    index=len(k)
    i=0
    days=0
    m=0
    result=[]
    while(i<=index-1):
        while((k[i]==1)&(i<=index-1)):
            days=days+1
            i=i+1
            m=days
            if(i==index):
                break
        if(m>0):        
            result.append(m+1)
        m=0
        days=0
        i+=1
    r=np.array(result)
    length=len(r)
    res=[]
    if(length<=1):
        return 0
    if(length==2):
        res.append(r[0])
        res.append(r[1])
        res=np.array(res)
        var=res.var()
        return var
    if(length==3):
        m=(r[0]+r[1])/2
        res.append(m)
        res.append(r[2])
        res=np.array(res)
        var=res.var()
        return var
    if(length>=4):
        p=int((length-1)*0.75)
        sum1=0
        sum2=0
        for i in range(0,p,1):
            sum1=sum1+r[i]
        for j in range(p,length,1):
            sum2=sum2+r[j]
        avg1=np.mean(sum1)
        avg2=np.mean(sum2)
        res.append(avg1)
        res.append(avg2)
        res=np.array(res)
        var=res.var()
        return var

In [13]:
def launch_extract(l,df):
    max_lx=l.groupby('user_id').agg({'day':get_max_lx}).reset_index().rename(columns={'day':'max_lx_days'})
    last_lx=l.groupby('user_id').agg({'day':get_last_lx}).reset_index().rename(columns={'day':'last_lx_days'})
    lx_var=l.groupby('user_id').agg({'day':get_lx_var}).reset_index().rename(columns={'day':'lx_var'})
    launch=l.sort_values(by=['user_id','day'],ascending=True)
    diff_days=launch.groupby('user_id')['day'].diff().rename(columns={'day':'diff_days'})
    new_launch=pd.concat([launch,diff_days],axis=1)
    new_launch=new_launch.rename(columns={0:'diff_days'})
    new_launch=new_launch.fillna(0)
    gap_times=new_launch.groupby('user_id').agg({'diff_days':get_gap_times}).reset_index().rename(columns={'diff_days':'gap_times'})
    max_gap=new_launch.groupby('user_id').agg({'diff_days':np.max}).reset_index().rename(columns={'diff_days':'max_gap'})
    ######
    lastlx_var=l.groupby('user_id').agg({'day':last_lx_var}).reset_index().rename(columns={'day':'last_lx_var'})
    quarterlx_var=l.groupby('user_id').agg({'day':quarter_lx_var}).reset_index().rename(columns={'day':'quarter_lx_var'})
    df1=pd.merge(df,max_lx,how='left',on=['user_id'])
    df2=pd.merge(df1,last_lx,how='left',on=['user_id'])
    df3=pd.merge(df2,lx_var,how='left',on=['user_id'])
    df4=pd.merge(df3,gap_times,how='left',on=['user_id'])
    df5=pd.merge(df4,max_gap,how='left',on=['user_id'])
    df6=pd.merge(df5,lastlx_var,how='left',on=['user_id'])
    dataframe=pd.merge(df6,quarterlx_var,how='left',on=['user_id'])
    dataframe=dataframe.fillna(0)
    return dataframe

In [14]:
def add_launch_feature(df):
    max_lx_launch=df['max_lx_days']/df['launch_days']
    max_lx_p=df['max_lx_days']/df['period']
    lastgap_maxlx=df['latest_gap']/df['max_lx_days']
    lastgap_lastlx=df['latest_gap']/df['last_lx_days']
    avg_lx=df['launch_days']/df['gap_times']
    lastgap_avglx=df['latest_gap']/avg_lx
    #
    maxgap_empty=df['max_gap']/df['empty_days']
    maxgap_p=df['max_gap']/df['period']
    max_gap_lx=df['max_gap']/df['max_lx_days']
    lastgap_maxgap=df['latest_gap']/df['max_gap']
    avg_gap=df['empty_days']/df['gap_times']
    avg_gaplx_rate=avg_gap/avg_lx
    #
    lastlx_maxlx=df['last_lx_days']/df['max_lx_days']
    lastlx_avglx=df['last_lx_days']/avg_lx
    lastgap_avggap=df['latest_gap']/avg_gap
    #
    last_quarter_lxvar=df['last_lx_var']/df['quarter_lx_var']
    df.insert(62,'max_lx_launch',max_lx_launch)
    df.insert(63,'max_lx_p',max_lx_p)
    df.insert(64,'lastgap_maxlx',lastgap_maxlx)
    df.insert(65,'lastgap_lastlx',lastgap_lastlx)
    df.insert(66,'avg_lx',avg_lx)
    df.insert(67,'lastgap_avglx',lastgap_avglx)
    df.insert(68,'maxgap_empty',maxgap_empty)
    df.insert(69,'maxgap_p',maxgap_p)
    df.insert(70,'max_gap_lx',max_gap_lx)
    df.insert(71,'lastgap_maxgap',lastgap_maxgap)
    df.insert(72,'avg_gap',avg_gap)
    df.insert(73,'avg_gaplx_rate',avg_gaplx_rate)
    df.insert(74,'lastlx_maxlx',lastlx_maxlx)
    df.insert(75,'lastlx_avglx',lastlx_avglx)
    df.insert(76,'lastgap_avggap',lastgap_avggap)
    df.insert(77,'last_quarter_lxvar',last_quarter_lxvar)
    isinf=np.isinf(df)
    df[isinf]=0
    df=df.fillna(0)
    return df

In [15]:
def get_last_ratio(a):
    a=np.array(a)
    last=len(a)-1
    last_n=a[last]
    mean_n=a.mean()
    ratio=last_n/mean_n
    return ratio
def get_last_var(a):
    a=np.array(a)
    last=len(a)-1
    last_n=a[last]
    mean_n=a.mean()
    r=[]
    r.append(last_n)
    r.append(mean_n)
    r=np.array(r)
    var=np.var(r)
    return var
def quarter_ratio(a):
    a=np.array(a)
    length=len(a)
    if(length==1):
        return 1
    if(length==2):
        r=a[1]/a[0]
        return r
    if(length==3):
        avg=(a[0]+a[1])/2
        r=a[2]/avg
        return r
    if(length>=4):
        p=int((length-1)*0.75)
        sum1=0
        sum2=0
        for i in range(0,p,1):
            sum1=sum1+a[i]
        for j in range(p,length,1):
            sum2=sum2+a[j]
        avg1=np.mean(sum1)
        avg2=np.mean(sum2)
        r=avg2/avg1
        return r
def quarter_var(a):
    a=np.array(a)
    length=len(a)
    r=[]
    if(length==1):
        return 0
    if(length==2):
        r.append(a[0])
        r.append(a[1])
        r=np.array(r)
        var=np.var(r)
        return var
    if(length==3):
        avg=(a[0]+a[1])/2
        r.append(avg)
        r.append(a[2])
        r=np.array(r)
        var=np.var(r)
        return var
    if(length>=4):
        p=int((length-1)*0.75)
        sum1=0
        sum2=0
        for i in range(0,p,1):
            sum1=sum1+a[i]
        for j in range(p,length,1):
            sum2=sum2+a[j]
        avg1=np.mean(sum1)
        avg2=np.mean(sum2)
        r.append(avg1)
        r.append(avg2)
        r=np.array(r)
        var=np.var(r)
        return var

In [16]:
def add_var_ratio(df,v,a):
    new_v=v.sort_values(by=['user_id','day'],ascending=True)
    daily_vt=new_v.groupby(['user_id','day']).size().reset_index().rename(columns={0:'v_times'})
    #mean_vtimes=daily_vt.groupby('user_id').agg({'v_times':np.mean}).reset_index().rename(columns={'v_times':'mean_vtimes'})
    last_v_ratio=daily_vt.groupby('user_id').agg({'v_times':get_last_ratio}).reset_index().rename(columns={'v_times':'last_v_ratio'})
    quarter_v_r=daily_vt.groupby('user_id').agg({'v_times':quarter_ratio}).reset_index().rename(columns={'v_times':'quarter_v_r'})
    v_var=daily_vt.groupby('user_id')['v_times'].var().reset_index().rename(columns={'v_times':'v_var'}).fillna(0)
    vt_last_var=daily_vt.groupby('user_id').agg({'v_times':get_last_var}).reset_index().rename(columns={'v_times':'vt_last_var'})
    vt_quarter_var=daily_vt.groupby('user_id').agg({'v_times':quarter_var}).reset_index().rename(columns={'v_times':'vt_quarter_var'})
    #########每日行为
    new_a=a.sort_values(by=['user_id','day'],ascending=True)
    daily_anums=new_a.groupby(['user_id','day']).size().reset_index().rename(columns={0:'a_nums'})
    last_a_ratio=daily_anums.groupby('user_id').agg({'a_nums':get_last_ratio}).reset_index().rename(columns={'a_nums':'last_a_ratio'})
    quarter_a_r=daily_anums.groupby('user_id').agg({'a_nums':quarter_ratio}).reset_index().rename(columns={'a_nums':'quarter_a_r'})
    a_var=daily_anums.groupby('user_id')['a_nums'].var().reset_index().rename(columns={'a_nums':'a_var'}).fillna(0)
    an_last_var=daily_anums.groupby('user_id').agg({'a_nums':get_last_var}).reset_index().rename(columns={'a_nums':'an_last_var'})
    an_quarter_var=daily_anums.groupby('user_id').agg({'a_nums':quarter_var}).reset_index().rename(columns={'a_nums':'an_quarter_var'})
    #########视频id
    video_ids=new_a.groupby(['user_id','day'])['video_id'].nunique().reset_index().rename(columns={'video_id':'video_ids'})
    last_vid_ratio=video_ids.groupby('user_id').agg({'video_ids':get_last_ratio}).reset_index().rename(columns={'video_ids':'last_vid_ratio'})
    quarter_vid_r=video_ids.groupby('user_id').agg({'video_ids':quarter_ratio}).reset_index().rename(columns={'video_ids':'quarter_vid_r'})
    ids_last_var=video_ids.groupby('user_id').agg({'video_ids':get_last_var}).reset_index().rename(columns={'video_ids':'ids_last_var'})
    ids_quarter_var=video_ids.groupby('user_id').agg({'video_ids':quarter_var}).reset_index().rename(columns={'video_ids':'ids_quarter_var'})
    vid_var=video_ids.groupby('user_id')['video_ids'].var().reset_index().rename(columns={'video_ids':'vid_var'}).fillna(0)
    #########作者id
    a_ids=new_a.groupby(['user_id','day'])['author_id'].nunique().reset_index().rename(columns={'author_id':'a_ids'})
    last_aid_ratio=a_ids.groupby('user_id').agg({'a_ids':get_last_ratio}).reset_index().rename(columns={'a_ids':'last_aid_ratio'})
    quarter_aid_r=a_ids.groupby('user_id').agg({'a_ids':quarter_ratio}).reset_index().rename(columns={'a_ids':'quarter_aid_r'})
    aid_last_var=a_ids.groupby('user_id').agg({'a_ids':get_last_var}).reset_index().rename(columns={'a_ids':'aid_last_var'})
    aid_quarter_var=a_ids.groupby('user_id').agg({'a_ids':quarter_var}).reset_index().rename(columns={'a_ids':'aid_quarter_var'})
    aid_var=a_ids.groupby('user_id')['a_ids'].var().reset_index().rename(columns={'a_ids':'aid_var'})
    ############
    df=pd.merge(df,last_v_ratio,how='left',on=['user_id'])
    df=pd.merge(df,quarter_v_r,how='left',on=['user_id'])
    df=pd.merge(df,v_var,how='left',on=['user_id'])
    df=pd.merge(df,vt_last_var,how='left',on=['user_id'])
    df=pd.merge(df,vt_quarter_var,how='left',on=['user_id'])
    ############
    df=pd.merge(df,last_a_ratio,how='left',on=['user_id'])
    df=pd.merge(df,quarter_a_r,how='left',on=['user_id'])
    df=pd.merge(df,a_var,how='left',on=['user_id'])
    df=pd.merge(df,an_last_var,how='left',on=['user_id'])
    df=pd.merge(df,an_quarter_var,how='left',on=['user_id'])
    ############
    df=pd.merge(df,last_vid_ratio,how='left',on=['user_id'])
    df=pd.merge(df,quarter_vid_r,how='left',on=['user_id'])
    df=pd.merge(df,ids_last_var,how='left',on=['user_id'])
    df=pd.merge(df,ids_quarter_var,how='left',on=['user_id'])
    df=pd.merge(df,vid_var,how='left',on=['user_id'])
    ############
    df=pd.merge(df,last_aid_ratio,how='left',on=['user_id'])
    df=pd.merge(df,quarter_aid_r,how='left',on=['user_id'])
    df=pd.merge(df,aid_last_var,how='left',on=['user_id'])
    df=pd.merge(df,aid_quarter_var,how='left',on=['user_id'])
    df=pd.merge(df,aid_var,how='left',on=['user_id'])
    df=df.fillna(0)
    return df

In [17]:
def insert_var(df):
    last_quarter_vr=df['last_v_ratio']/df['quarter_v_r']
    last_quarter_vvar=df['vt_last_var']/df['vt_quarter_var']
    last_quarter_ar=df['last_a_ratio']/df['quarter_a_r']
    last_quarter_avar=df['an_last_var']/df['an_quarter_var']
    last_quarter_vidr=df['last_vid_ratio']/df['quarter_vid_r']
    last_quarter_vidvar=df['ids_last_var']/df['aid_quarter_var']
    last_quarter_aidr=df['last_aid_ratio']/df['quarter_aid_r']
    last_quarter_aidv=df['aid_last_var']/df['aid_quarter_var']
    df.insert(98,'last_quarter_vr',last_quarter_vr)
    df.insert(99,'last_quarter_vvar',last_quarter_vvar)
    df.insert(100,'last_quarter_ar',last_quarter_ar)
    df.insert(101,'last_quarter_avar',last_quarter_avar)
    df.insert(102,'last_quarter_vidr',last_quarter_vidr)
    df.insert(103,'last_quarter_vidvar',last_quarter_vidvar)
    df.insert(104,'last_quarter_aidr',last_quarter_aidr)
    df.insert(105,'last_quarter_aidv',last_quarter_aidv)
    isinf=np.isinf(df)
    df[isinf]=0
    df=df.fillna(0)
    return df

In [18]:
def labeled(data,test_user_id):
    #调用方法前必须保证data已经有了label列，并且值都为0
    ind=data.index.max()+1
    label=np.zeros((ind,1))
    data.insert(106,'label',label)
    label_index=data[data['user_id'].isin(test_user_id)].index
    for x in label_index:
        data.set_value(index=x,col='label',value=1)
    return data

In [19]:
r=pd.read_csv(r'C:\Users\hp\Desktop\CompetitionData\B\user_register_log.csv')
l=pd.read_csv(r'C:\Users\hp\Desktop\CompetitionData\B\app_launch_log.csv')
a=pd.read_csv(r'C:\Users\hp\Desktop\CompetitionData\B\user_activity_log.csv')
v=pd.read_csv(r'C:\Users\hp\Desktop\CompetitionData\B\video_create_log.csv')

########feature_create(r,l,v,a)
########feature_generate(df)/generate_test(df)
########train_add_feature(df)/test_add_feature(df)
########launch_extract(l,df)
########add_launch_feature(df)
########add_var_ratio(df,v,a)
########insert_var(df)
########labeled(data,test_user_id)

In [20]:
r_train,r_test,l_train,l_test,v_train,v_test,a_train,a_test=split_train_test(r,l,v,a)

In [21]:
df1=feature_create(r_train,l_train,v_train,a_train)

In [22]:
df2=feature_generate(df1)

In [23]:
df2=train_add_feature(df2)

In [24]:
df3=launch_extract(l_train,df2)

In [25]:
df4=add_launch_feature(df3)

In [26]:
df5=add_var_ratio(df4,v_train,a_train)

In [27]:
df6=insert_var(df5)

In [28]:
test_user_id=user_id_concat(r_test,l_test,v_test,a_test)
test_user_id=test_user_id['user_id']
test_user_id.shape

(29529,)

In [29]:
df=labeled(df6,test_user_id)
df.describe()

,user_id,launch_days,video_times,act_nums,act_video_nums,act_authors,register_day,register_type,latest_day,types_avg_video,...,aid_var,last_quarter_vr,last_quarter_vvar,last_quarter_ar,last_quarter_avar,last_quarter_vidr,last_quarter_vidvar,last_quarter_aidr,last_quarter_aidv,label
count,4.031400e+04,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,...,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000,40314.000000
mean,6.924936e+05,4.163963,0.610334,23.816242,17.640100,14.769013,13.957012,0.987275,19.028873,0.510374,...,133.807153,0.148372,0.012417,0.570243,0.127691,0.566424,0.256374,0.572156,0.114539,0.455499
std,3.991940e+05,4.504099,3.046827,71.256529,51.865335,44.734824,6.976685,1.255440,6.148709,0.562364,...,966.526696,0.391641,0.250421,0.769576,2.921550,0.752682,6.704953,0.745571,5.294635,0.498022
min,8.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.476635e+05,1.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,15.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.958855e+05,2.000000,0.000000,0.000000,0.000000,0.000000,14.000000,1.000000,22.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.037664e+06,5.000000,0.000000,11.000000,8.000000,6.000000,20.750000,1.000000,24.000000,1.000000,...,0.333333,0.000000,0.000000,1.000000,0.009990,1.000000,0.000244,1.000000,0.002532,1.000000
max,1.383717e+06,24.000000,103.000000,2122.000000,1067.000000,1004.000000,24.000000,11.000000,24.000000,3.000000,...,51200.000000,6.636986,43.183673,8.551965,420.250000,8.072727,1225.000000,7.633929,1008.062500,1.000000


In [30]:
from collections import Counter
Counter(df['label'])

Counter({0.0: 12773, 1.0: 9569})

In [30]:
df.to_csv(r'C:\Users\hp\Desktop\CompetitionData\B\TrainData24DAYS105cols.csv')

##################TestData

In [31]:
data=feature_create(r,l,v,a)

In [32]:
data=generate_test(data)

In [33]:
data=test_add_feature(data)

In [34]:
data=launch_extract(l,data)

In [35]:
data=add_launch_feature(data)

In [36]:
data=add_var_ratio(data,v,a)

In [37]:
data=insert_var(data)

In [38]:
data.describe()

,user_id,launch_days,video_times,act_nums,act_video_nums,act_authors,register_day,register_type,latest_day,types_avg_video,...,aid_quarter_var,aid_var,last_quarter_vr,last_quarter_vvar,last_quarter_ar,last_quarter_avar,last_quarter_vidr,last_quarter_vidvar,last_quarter_aidr,last_quarter_aidv
count,5.148000e+04,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,...,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000,51480.000000
mean,6.919959e+05,4.904740,0.696368,20.368590,15.148174,12.475932,16.905167,0.981838,23.414025,0.496961,...,355.406624,95.192793,0.158822,0.012515,0.576305,0.106828,0.572723,0.209863,0.577061,0.074478
std,3.993951e+05,5.568106,3.611903,65.732924,47.829807,40.805280,8.371232,1.277424,7.644914,0.555849,...,3263.712655,790.278105,0.406720,0.186392,0.808130,1.721772,0.789499,3.234549,0.771544,1.464565
min,8.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.467558e+05,1.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,20.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.950660e+05,2.000000,0.000000,0.000000,0.000000,0.000000,18.000000,1.000000,27.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.037541e+06,6.000000,0.000000,9.000000,6.000000,5.000000,24.000000,1.000000,30.000000,1.000000,...,0.000000,0.200000,0.000000,0.000000,1.000000,0.004589,1.000000,0.000000,1.000000,0.000000
max,1.383717e+06,30.000000,206.000000,2122.000000,1090.000000,1019.000000,30.000000,11.000000,30.000000,3.000000,...,252004.000000,51200.000000,4.714286,27.562500,9.200108,173.361111,8.341667,462.250000,8.068966,232.562500


In [39]:
data.to_csv(r'C:\Users\hp\Desktop\CompetitionData\B\TestData24DAYS105cols.csv')